# Toy Case

This case employs a simple reaction `A > B` performed in a batch reactor to introduce the provided modelling environment.  

<div><center><img src="https://upload.wikimedia.org/wikipedia/commons/1/15/Batch_reactor_STR.svg" width="150"/></center></div>

The mass balance equation for batch reactor is
$$\frac{dc}{dt} = r_r$$  
The reaction rate constant of `A > B` follows Arrhenius equation
$$k = A\cdot\exp{(-\frac{E_a}{RT})}$$
We reparameterise Arrhenius with reference to 20 oC
$$k = k_{ref}\cdot\exp{(-\frac{E_a}{R}(\frac{1}{T} - \frac{1}{293.15}))}$$
The reaction rate follows power law with respect to concentrations of reactants
$$r_r = k\prod c^n$$

In [1]:
# import required python libraries
import numpy as np
import pandas as pd
from bayes_opt import BayesianOptimization, acquisition
from plotly.offline import init_notebook_mode

from cases.toy_case import ToyCase

init_notebook_mode(connected=True)

# define phenomenon for the process to define the model
phenos = {
    "Mass accumulation":    "Batch",
    "Flow pattern":         "Well_Mixed",
    "Mass transport":       [],
}

toy_case = ToyCase(phenos, random_seed=42)

#### Parameter Setup
As modelling parameters are associated with different dimensions (solid/liquid, stream, reaction, species), all parameters are indicated as `(Parameter Name, Solid/Liquid index, Stream index, Reaction index, Species index)` in this provided modelling environment.  
Note that `None` will be applied if the parameter is not associated with that specific dimension.  

> **Operation parameters**
>
> - ("Temperature", None, None, None, None)     [No dimension related to temperature]
> - ("Batch_Time", None, None, None, None)      [No dimension related to batch time]
> - ("Concentration", None, 0, None, 0)         [Concentration of A]  
>  
> **Kinetics parameters**
> - ("Activation_Energy", None, 0, 0, None)                 [Activation energy is related to stream and reaction dimensions]
> - ("Referenced_Reaction_Rate_Constant", None, 0, 0, None) [Referenced reaction rate constant is related to stream and reaction dimensions]

#### Unit Setup

Parameter setup can be found in the method `var2unit`

| Parameter                         | Unit   |
| --------------------------------- | ------ |
| Referenced_Reaction_Rate_Constant |        |
| Activation_Energy                 | kJ/mol |
| Temperature                       | oC     |
| Concentration                     | mol/L  |
| Batch_Time                        | min    |

### Model Simulation
- Print model kinetics parameters and operation parameters
- Set operation condition example and plot the concentration curves against operation time
- Plot B concentration profiles at varied temperatures
- Plot B concentration profiles at varied initial A concentrations

In [2]:
# list of kinetics and operation parameters
toy_case.kinetics_params(), toy_case.operation_params()

({('Activation_Energy', None, 0, 0, None): 80.0,
  ('Referenced_Reaction_Rate_Constant', None, 0, 0, None): 0.0001,
  ('Stoichiometric_Coefficient', None, None, 0, 0): -1,
  ('Stoichiometric_Coefficient', None, None, 0, 1): 1,
  ('Partial_Order', None, None, 0, 0): 1},
 {('Concentration', None, 0, None, 0): None,
  ('Temperature', None, None, None, None): None,
  ('Batch_Time', None, None, None, None): None})

In [3]:
# model simulation and plot concentration profiles
# A concentration   [A] = 1 mol/L
# temperature       T = 50.0 oC
# batch time        t_b = 60.0 min
operation_params = {
    ('Concentration', None, 0, None, 0): 1.0,
    ('Temperature', None, None, None, None): 50.0,
    ('Batch_Time', None, None, None, None): 60.0
}
toy_case.plot_simulation_profiles(operation_params)

In [4]:
# B concentration profiles under vared temperatures
# set temperature to a list of values: 25.0, 50.0, 75.0, 100.0
operation_params = {
    ('Concentration', None, 0, None, 0): 1.0,
    ('Temperature', None, None, None, None): [25.0, 50.0, 75.0, 100.0],
    ('Batch_Time', None, None, None, None): 60.0
}
toy_case.plot_product_profile_with_temperatures(operation_params)

In [5]:
# B concentration profiles under vared A concentrations
# set A concentration to a list of values: [0.5, 1.0, 1.5, 2.0]
operation_params = {
    ('Concentration', None, 0, None, 0): [0.5, 1.0, 1.5, 2.0],
    ('Temperature', None, None, None, None): 50.0,
    ('Batch_Time', None, None, None, None): 60.0
}
toy_case.plot_product_profile_with_A_concs(operation_params)

### Model Calibration
How to design experiments to determine kinetics?
In this notebook, we use a simple grid-based experimental approach.  
Recall that  
$k = k_{ref}\cdot\exp{(-\frac{E_a}{R}(\frac{1}{T} - \frac{1}{293.15}))}$, $r_r = k\prod c^n$  
By varying residence time and initial A concentration, we can calibrate $k$ for each temperature  
By varying temperature, we can calibrate the Activation Energy $E_a$

In [6]:
# print short names for operation parameters
toy_case.operation_name2ind()

{'A_conc': ('Concentration', None, 0, None, 0),
 'temp': ('Temperature', None, None, None, None),
 't_b': ('Batch_Time', None, None, None, None)}

In [7]:
# set up grid experiment data points
exp_data = {
    "A_conc": [0.25, 0.25, 0.25, 0.25, 0.75, 0.75, 0.75, 0.75],
    "temp":   [  25,   25,   50,   50,   25,   25,   50,   50],
    "t_b":    [  30,   60,   30,   60,   30,   60,   30,   60],
}
# create tabular dataset
exp_dataset = pd.DataFrame(exp_data)
# run simulation
exp_dataset = toy_case.run_dataset(exp_dataset)
exp_dataset

,A_conc,temp,t_b,outlet_B_conc
0,0.25,25,30,0.067028
1,0.25,25,60,0.116086
2,0.25,50,30,0.244347
3,0.25,50,60,0.249872
4,0.75,25,30,0.201084
5,0.75,25,60,0.348266
6,0.75,50,30,0.733037
7,0.75,50,60,0.749616


### Prediction error minimisation
Assume these data are obtained from experiments. Then, the model is fitted by minimising the prediction error on these operation conditions  
$Error = Mean((c_{measured,B} - c_{predicted,B})^2)$  
The quasi-Newton algorithm `L-BFGS-B` is used to minimise this error  
Find original codes of the `calibrate` method in `cases/toy_case.py` file

In [8]:
# Run the `calibrate` method
# Fitted activation energy and referenced reaction rate constant are close to the actual values
cal_param_bounds = {
    ("Activation_Energy", None, 0, 0, None):                    (40, 120), # kJ/mol
    ("Referenced_Reaction_Rate_Constant", None, 0, 0, None):    (1e-6, 1e-3),
}
toy_case.calibrate(cal_param_bounds, exp_dataset)

{('Activation_Energy', None, 0, 0, None): 79.41151,
 ('Referenced_Reaction_Rate_Constant', None, 0, 0, None): 0.000101}

In [9]:
# print the ground-truth kinetics
toy_case.kinetics_params()

{('Activation_Energy', None, 0, 0, None): 80.0,
 ('Referenced_Reaction_Rate_Constant', None, 0, 0, None): 0.0001,
 ('Stoichiometric_Coefficient', None, None, 0, 0): -1,
 ('Stoichiometric_Coefficient', None, None, 0, 1): 1,
 ('Partial_Order', None, None, 0, 0): 1}